## Entrainement de differents modèles pour calculer les probabilités de retour à l'emploi

In [1]:
from pyspark.sql import SparkSession

### Rechargement du dataset Personnes

In [2]:
# MASTER="yarn"           # Si Spark pushdown sur CML
MASTER="local[*]"       # Si execute parallele sur CML

spark = SparkSession.builder.appName("O_ingestion").master(MASTER).getOrCreate()


7.1.9 and 7.2.17 are the last CDP runtime releases where Spark 2 is supported.
Please migrate your Spark 2 applications to Spark 3.

Updating Spark 2 applications for Spark 3:
https://docs.cloudera.com/runtime/7.2.16/running-spark-applications/topics/spark-update-spark2-spark3.html



In [3]:
sdf=spark.sql("select * from olivier.personnes")

Hive Session ID = d6cedd98-b4fc-4910-9cf2-83c9e64183bd


### Initialisation des experimentation MLFLOW pour l'Atelier 1

In [6]:
import mlflow

In [7]:
mlflow.set_experiment ("Experimentations Atelier 1")

2024/03/12 17:33:34 INFO mlflow.tracking.fluent: Experiment with name 'Experimentations Atelier 1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/cdsw/.experiments/rgbu-b5q0-sqoq-olbo', creation_time=None, experiment_id='rgbu-b5q0-sqoq-olbo', last_update_time=None, lifecycle_stage='active', name='Experimentations Atelier 1', tags={}>

## Lancement d'une entrainement Regression logistique de Spark ML Lib

In [4]:
variables = sdf.columns[1:-1]   # Toutes les variables moins la premier (id) et la derniere qui est la variable à étudier

In [5]:
sdf.printSchema()

root
 |-- id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- sexe: string (nullable = true)
 |-- niveau_education: string (nullable = true)
 |-- duree_chomage: long (nullable = true)
 |-- experience_ant: long (nullable = true)
 |-- competence: string (nullable = true)
 |-- formation: boolean (nullable = true)
 |-- taux_chomage_local: double (nullable = true)
 |-- secteur_activite: string (nullable = true)
 |-- reseau_professionnel: long (nullable = true)
 |-- support_social: long (nullable = true)
 |-- retour_emploi: boolean (nullable = true)



#### Transformation des variables categorielles

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [7]:
sdf.dtypes[1:-1]

[('age', 'bigint'),
 ('sexe', 'string'),
 ('niveau_education', 'string'),
 ('duree_chomage', 'bigint'),
 ('experience_ant', 'bigint'),
 ('competence', 'string'),
 ('formation', 'boolean'),
 ('taux_chomage_local', 'double'),
 ('secteur_activite', 'string'),
 ('reseau_professionnel', 'bigint'),
 ('support_social', 'bigint')]

In [87]:
# Extract les colonnes categories
cat_cols=[]
feature_cols=[]
for colname, coltype in sdf.dtypes[1:-1]:
    if ( coltype == 'string'):
        cat_cols += [colname]
    else:
        feature_cols += [colname]


In [84]:
stages = [] 

# liste toutes les colonne, type de sdf moins la premiere (id) et derniere colonne (variable à etudier)

for colname in cat_cols:

   # Assigne un indice aux variable categorielle
   indexer = StringIndexer(inputCol=colname, outputCol=colname + "_index") 
        
   encoder = OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol=colname + "_vec")  

   # Enregistre les colonnes index à ajouter a la sdf
   stages       += [indexer, encoder]  

In [93]:
from pyspark.ml.feature import VectorAssembler

# Transform all features into a vector
feature_cols = ["carat", "depth", "table", "x", "y", "z"]
assemblerInputs = [c + "_vec" for c in cat_cols] + feature_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

# Create pipeline and use on dataset
pipeline = Pipeline(stages=stages)
)

In [94]:
sdf = pipeline.fit(sdf).transform(sdf)

IllegalArgumentException: 'Data type string of column sexe is not supported.\nData type string of column niveau_education is not supported.\nData type string of column competence is not supported.\nData type string of column secteur_activite is not supported.'

In [63]:
age = "25"
catégorie = "Jeune" if age == "20" else "Adulte"
print(catégorie)

Adulte


In [54]:
pipeline = Pipeline(stages=stages)
model = pipeline.fit(sdf).transform(sdf)
model.show()

+-----+---+-----+----------------+-------------+--------------+--------------------+---------+------------------+----------------+--------------------+--------------+-------------+
|   id|age| sexe|niveau_education|duree_chomage|experience_ant|          competence|formation|taux_chomage_local|secteur_activite|reseau_professionnel|support_social|retour_emploi|
+-----+---+-----+----------------+-------------+--------------+--------------------+---------+------------------+----------------+--------------------+--------------+-------------+
|74947| 18|homme|        DOCTORAT|           24|             7|Equality and dive...|     true|16.510081672408976|           Group|                  16|             1|        false|
|45820| 47|femme|         CAP/BEP|           20|             8|Scientist, physio...|    false|19.468590596632215|             Ltd|                   5|             1|         true|
|77624| 67|homme|    BACCALAUREAT|            0|            18| Corporate treasurer|    false|1

In [33]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

 
cat_cols= ["cut", "color", "clarity"]
stages = [] # Stages in Pipeline

for c in cat_cols:
    stringIndexer = StringIndexer(inputCol=c, outputCol=c + "_index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], \
            outputCols=[c + "_vec"])    
    stages += [stringIndexer, encoder] # Stages will be run later on

TypeError: __init__() got an unexpected keyword argument 'inputCols'

In [ ]:
from pyspark.ml.feature import VectorAssembler

#specify predictor variables
assembler = VectorAssembler(inputCols=['hours', 'prep_exams'], outputCol='features')

#format DataFrame for linear regression
data = assembler.transform(df)
data = data.select('features', 'score')